In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import mutual_info_score, 


In [2]:
df=pd.read_csv('bank-full.csv', delimiter=';')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [4]:
chosen_columns=['age','job','marital','education','balance','housing','contact',
                'day','month','duration','campaign','pdays','previous','poutcome','y']

In [5]:
df=df[chosen_columns]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   balance    45211 non-null  int64 
 5   housing    45211 non-null  object
 6   contact    45211 non-null  object
 7   day        45211 non-null  int64 
 8   month      45211 non-null  object
 9   duration   45211 non-null  int64 
 10  campaign   45211 non-null  int64 
 11  pdays      45211 non-null  int64 
 12  previous   45211 non-null  int64 
 13  poutcome   45211 non-null  object
 14  y          45211 non-null  object
dtypes: int64(7), object(8)
memory usage: 5.2+ MB


### Question 1
What is the most frequent observation (mode) for the column education?

In [7]:
df['education'].mode()

0    secondary
Name: education, dtype: object

In [8]:
df.select_dtypes(include=['int64', 'float64']).columns


Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'], dtype='object')

In [9]:
df.select_dtypes(include=['object']).columns

Index(['job', 'marital', 'education', 'housing', 'contact', 'month',
       'poutcome', 'y'],
      dtype='object')

In [10]:
numerical=['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
categorical=['job', 'marital', 'education', 'housing', 'contact', 'month','poutcome']

In [14]:
df[numerical].corr()

,age,balance,day,duration,campaign,pdays,previous
age,1.000000,0.097783,-0.009120,-0.004648,0.004760,-0.023758,0.001288
balance,0.097783,1.000000,0.004503,0.021560,-0.014578,0.003435,0.016674
day,-0.009120,0.004503,1.000000,-0.030206,0.162490,-0.093044,-0.051710
duration,-0.004648,0.021560,-0.030206,1.000000,-0.084570,-0.001565,0.001203
campaign,0.004760,-0.014578,0.162490,-0.084570,1.000000,-0.088628,-0.032855
pdays,-0.023758,0.003435,-0.093044,-0.001565,-0.088628,1.000000,0.454820
previous,0.001288,0.016674,-0.051710,0.001203,-0.032855,0.454820,1.000000


## Question 2
Create the correlation matrix for the numerical features of your dataset. I
n a correlation matrix, you compute the correlation coefficient between every pair of features.

  What are the two features that have the biggest correlation?  

age and balance  
day and campaign  
day and pdays  
pdays and previous

In [16]:
np.corrcoef(df.age, df.balance)

array([[1.        , 0.09778274],
       [0.09778274, 1.        ]])

In [19]:
np.corrcoef(df.day, df.campaign)

array([[1.        , 0.16249022],
       [0.16249022, 1.        ]])

In [21]:
np.corrcoef(df.day, df.pdays)

array([[ 1.        , -0.09304407],
       [-0.09304407,  1.        ]])

In [22]:
np.corrcoef(df.pdays, df.previous)

array([[1.        , 0.45481964],
       [0.45481964, 1.        ]])

In [24]:
### answer: pdays and previouse

### Target encoding
Now we want to encode the y variable. 
Let's replace the values yes/no with 1/0.  


In [40]:
# target encoding
df.y=df.y.replace({'yes': 1, 'no': 0})

In [41]:
df.y

0        0
1        0
2        0
3        0
4        0
        ..
45206    1
45207    1
45208    1
45209    0
45210    0
Name: y, Length: 45211, dtype: int64

### Split the data  
Split your data in train/val/test sets with 60%/20%/20% distribution.   
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.  
Make sure that the target value y is not in your dataframe. 

In [44]:
from sklearn.model_selection import train_test_split
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [45]:
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=11)

In [47]:
y_train = df_train.y.values
y_val = df_val.y.values

In [49]:
del df_train['y']
del df_val['y']

In [50]:
len(df_train), len(df_val),len(df_test)

(27126, 9042, 9043)

### Question 3
Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only. 
Round the scores to 2 decimals using round(score, 2). 
Which of these variables has the biggest mutual information score? 

contact  
education   
housing  
poutcome  

In [55]:
def calculate_mi(series):
    return mutual_info_score(series, df_train_full.y)

df_mi = df_train_full[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')


display(df_mi.head())
display(df_mi.tail())


,MI
poutcome,0.028893
month,0.024718
contact,0.013041
housing,0.009551
job,0.008437


,MI
contact,0.013041
housing,0.009551
job,0.008437
education,0.002564
marital,0.001980


### Question 4  
Now let's train a logistic regression.  
Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.  
Fit the model on the training dataset.  
To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:  
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)  
Calculate the accuracy on the validation dataset and round it to 2 decimal digits.  
What accuracy did you get?  

0.6  
0.7  
0.8  
0.9  

In [56]:
from sklearn.feature_extraction import DictVectorizer

In [65]:
#df_train[categorical +numerical]

In [58]:
train_dict = df_train.to_dict(orient='records')

In [60]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')

In [61]:
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

DictVectorizer(sparse=False)

In [62]:
X_train = dv.transform(train_dict)

In [63]:
X_train.shape

(27126, 47)

In [64]:
from sklearn.linear_model import LogisticRegression

In [66]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [68]:
val_dict = df_val[categorical + numerical].to_dict(orient='records')

In [69]:
X_val = dv.transform(val_dict)

In [70]:
X_val.shape

(9042, 47)

In [89]:
y_pred=model.predict(X_val)
round(model.score(X_val, y_val),2)

0.9

### Question 5
Let's find the least useful feature using the feature elimination technique.  
Now exclude each feature from this set and train a model without it. Record the accuracy for each model.  
For each feature, calculate the difference between the original accuracy and the accuracy without the feature.  
Which of following feature has the smallest difference?  

age  
balance  
marital  
previous  

In [85]:
df_t=df_train[categorical + numerical]
df_v=df_val[categorical + numerical]
del df_t['age']
del df_v['age']
def train_model(df):
    train_dict = df_t.to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    dv.fit(train_dict)
    X_train = dv.transform(train_dict)
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    val_dict = df_v.to_dict(orient='records')
    X_val = dv.transform(val_dict)
    s=model.score(X_val, y_val)
    return s
    
train_model(df_t)-0.8971466489714665

-0.00022119000221187957

In [84]:
df_t=df_train[categorical + numerical]
df_v=df_val[categorical + numerical]
del df_t['balance']
del df_v['balance']
def train_model(df):
    train_dict = df_t.to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    dv.fit(train_dict)
    X_train = dv.transform(train_dict)
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    val_dict = df_v.to_dict(orient='records')
    X_val = dv.transform(val_dict)
    s=model.score(X_val, y_val)
    return s
    
train_model(df_t)-0.8971466489714665  

0.0002211900022119906

In [86]:
df_t=df_train[categorical + numerical]
df_v=df_val[categorical + numerical]
del df_t['marital']
del df_v['marital']
def train_model(df):
    train_dict = df_t.to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    dv.fit(train_dict)
    X_train = dv.transform(train_dict)
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    val_dict = df_v.to_dict(orient='records')
    X_val = dv.transform(val_dict)
    s=model.score(X_val, y_val)
    return s
    
train_model(df_t)-0.8971466489714665

0.0

In [87]:
df_t=df_train[categorical + numerical]
df_v=df_val[categorical + numerical]
del df_t['previous']
del df_v['previous']
def train_model(df):
    train_dict = df_t.to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    dv.fit(train_dict)
    X_train = dv.transform(train_dict)
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    val_dict = df_v.to_dict(orient='records')
    X_val = dv.transform(val_dict)
    s=model.score(X_val, y_val)
    return s
    
train_model(df_t)-0.8971466489714665

-0.00011059500110588427

In [88]:
## answer:marital

###  Question 6
Now let's train a regularized logistic regression.
Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
Train models using all the features as in Q4.
Calculate the accuracy on the validation dataset and round it to 3 decimal digits.
Which of these C leads to the best accuracy on the validation set?

0.01
0.1
1
10
100
Note: If there are multiple options, select the smallest C.

In [97]:


def train_model_regula(df):
    result=[]
    train_dict = df.to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    dv.fit(train_dict)
    X_train = dv.transform(train_dict)
    for c in [0.01, 0.1, 1, 10, 100]:    
        model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
        model.fit(X_train, y_train)
        val_dict = df_val.to_dict(orient='records')
        X_val = dv.transform(val_dict)
        s=round(model.score(X_val, y_val),3)
        result.append((c,s))
    return result
    
train_model_regula(df_train)

[(0.01, 0.895), (0.1, 0.896), (1, 0.897), (10, 0.897), (100, 0.897)]